# Выбор локации для скважины

Мы выполняем задачу бизнеса "ГлавРосГосНефть.


Цель:
* Определить регион для бурения новой скважины.

Нам доступны файлы с пробами нефти в трех регионах. В каждом - 10 000 месторождений, где измерили качество и объем её запасов.
Необходимо построить модель машинного обучения, помогающая определить регион, где добыча принесет наибольшую прибыль.

Шаги, которое необходимо выполнить для выбора локации:
- В избранном регионе найти месторождения, для каждого определить значения признаков;
- Построить модель и оценить объём запасов;
- Выбрать месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных

Данные геологоразведки трёх регионов находятся в файлах:

* /datasets/geo_data_0.csv.
* /datasets/geo_data_1.csv.
* /datasets/geo_data_2.csv.


* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
# Импортируем все необходимые библиотеки

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from IPython.display import display
import warnings

In [2]:
# Константы

# Бюджет
BUDGET = 10_000_000_000/200

# Сколько нужно тысяч баррелей для того, чтобы окупить затраты
COUNT_BARR = BUDGET/450000

In [3]:
# Сохраняем информацию файлов в датафреймы и выводим на экран первые 5 строк каждого

data0=pd.read_csv('/datasets/geo_data_0.csv')
data1=pd.read_csv('/datasets/geo_data_1.csv')
data2=pd.read_csv('/datasets/geo_data_2.csv')

display(data0.head())
display(data1.head())
display(data2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
# Изучаем общую информацию каждого датафрейма

display(data0.info())
display(data1.info())
display(data2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
# Получаем сводку числовых значений для каждого нашего датафрейма

display(data0.describe())
display(data1.describe())
display(data2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [6]:
# Проверяем наличие явных дубликатов

display(data0.duplicated().sum())
display(data1.duplicated().sum())
display(data2.duplicated().sum())

0

0

0

In [7]:
display(data0.nunique())
display(data1.nunique())
display(data2.nunique())

id          99990
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

id          99996
f0         100000
f1         100000
f2         100000
product        12
dtype: int64

id          99996
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

In [8]:
data0['id'].value_counts()

QcMuo    2
74z30    2
A5aEY    2
HZww2    2
Tdehs    2
        ..
tJixR    1
OUYI3    1
iXvMn    1
xd1Kt    1
V3PKm    1
Name: id, Length: 99990, dtype: int64

In [9]:
data0[data0['id'] == 'bxg6G']

,id,f0,f1,f2,product
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798


Пропущенных значений нет, дубликатов нет, стобцы имеют корректное наименование. Смущают лишь повторяющиеся значения в столбце id, но, т.к. данные в строках разные - убирать их не будем. Уберем лишь сам столбец с id, т.к. он не нужен для обучения модели.

Можно приступать к работе.

In [10]:
# Посмотрим на корреляцию признаков

display(data0.corr())
display(data1.corr())
display(data2.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


По полученным значениям можно сделать вывод:
* f0 и f1 практически никак не коррелируют
* f2 у первого и третьего региона имеет слабую корреляционную связь
* f2 у второго региона показывает очень сильную корреляцию с объемами запасов в скважине

## Обучение и проверка модели

In [11]:
# Начинаем делить данные на две выборки - обучающую и валидационную в соотношении 75:25

data0_train, data0_valid = train_test_split(data0, test_size=0.25, random_state=12345)
data1_train, data1_valid = train_test_split(data1, test_size=0.25, random_state=12345)
data2_train, data2_valid = train_test_split(data2, test_size=0.25, random_state=12345)

In [12]:
# Создаем переменные с признаками и переменные с целевым признаком

features_train0 = data0_train.drop(['id', 'product'], axis=1)
target_train0 = data0_train['product']
features_valid0 = data0_valid.drop(['id', 'product'], axis=1)
target_valid0 = data0_valid['product']

features_train1 = data1_train.drop(['id', 'product'], axis=1)
target_train1 = data1_train['product']
features_valid1 = data1_valid.drop(['id', 'product'], axis=1)
target_valid1 = data1_valid['product']

features_train2 = data2_train.drop(['id', 'product'], axis=1)
target_train2 = data2_train['product']
features_valid2 = data2_valid.drop(['id', 'product'], axis=1)
target_valid2 = data2_valid['product']

In [13]:
# Проверяем методом shape

print('data0_train:', features_train0.shape)
print(target_train0.shape)
print('data0_valid', features_valid0.shape)
print(target_valid0.shape)
print()
print('data1_train:', features_train1.shape)
print(target_train1.shape)
print('data1_valid', features_valid1.shape)
print(target_valid1.shape)
print()
print('data2_train:', features_train2.shape)
print(target_train2.shape)
print('data2_valid', features_valid2.shape)
print(target_valid2.shape)

data0_train: (75000, 3)
(75000,)
data0_valid (25000, 3)
(25000,)

data1_train: (75000, 3)
(75000,)
data1_valid (25000, 3)
(25000,)

data2_train: (75000, 3)
(75000,)
data2_valid (25000, 3)
(25000,)


In [14]:
# Первый регион

model = LinearRegression() # Создаем модель линейной регрессии
model.fit(features_train0,target_train0) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid0) # Предсказания модели

# Считаем средний предсказанный запас сырья 
data0_pred_mean = predictions_valid.mean()

# Считаем RMSE
result = mean_squared_error(target_valid0,predictions_valid)**0.5

# предсказания модели на валидационной выборке и сохраняем результат
# в отдельной переменной
data0_predictions_valid = model.predict(features_valid0)

print('Средний предсказанный запас сырья', data0_pred_mean)
print('Средний фактический запас сырья', target_valid0.mean())
print('RMSE модели на валидационной выборке:', result)

Средний предсказанный запас сырья 92.59256778438035
Средний фактический запас сырья 92.07859674082927
RMSE модели на валидационной выборке: 37.5794217150813


In [15]:
#Второй регион

model = LinearRegression()
model.fit(features_train1,target_train1)
predictions_valid = model.predict(features_valid1)
data1_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid1,predictions_valid)**0.5
data1_predictions_valid = model.predict(features_valid1)

print('Средний предсказанный запас сырья', data1_pred_mean)
print('Средний фактический запас сырья', target_valid1.mean())
print('RMSE модели на валидационной выборке:', result)

Средний предсказанный запас сырья 68.728546895446
Средний фактический запас сырья 68.72313602435997
RMSE модели на валидационной выборке: 0.893099286775617


In [16]:
#Третий регион

model = LinearRegression()
model.fit(features_train2,target_train2)
predictions_valid = model.predict(features_valid2)
data2_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid2,predictions_valid)**0.5
data2_predictions_valid = model.predict(features_valid2)

print('Средний предсказанный запас сырья', data2_pred_mean)
print('Средний фактический запас сырья', target_valid2.mean())
print('RMSE модели на валидационной выборке:', result)

Средний предсказанный запас сырья 94.96504596800489
Средний фактический запас сырья 94.88423280885438
RMSE модели на валидационной выборке: 40.02970873393434


In [17]:
def LR (features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    data_pred_mean = predictions_valid.mean()
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    data_predictions_valid = model.predict(features_valid)
    
    print('Средний предсказанный запас сырья', data_pred_mean)
    print('Средний фактический запас сырья', target_valid.mean())
    print('RMSE модели на валидационной выборке:', result)
    return

In [18]:
print(LR(features_train0, target_train0, features_valid0, target_valid0))

Средний предсказанный запас сырья 92.59256778438035
Средний фактический запас сырья 92.07859674082927
RMSE модели на валидационной выборке: 37.5794217150813
None


In [19]:
print(LR(features_train1, target_train1, features_valid1, target_valid1))

Средний предсказанный запас сырья 68.728546895446
Средний фактический запас сырья 68.72313602435997
RMSE модели на валидационной выборке: 0.893099286775617
None


In [20]:
print(LR(features_train2, target_train2, features_valid2, target_valid2))

Средний предсказанный запас сырья 94.96504596800489
Средний фактический запас сырья 94.88423280885438
RMSE модели на валидационной выборке: 40.02970873393434
None


Основываясь на полученных данных, мы видим, что у второго региона средний запас сырья сильно уступает двум другим, но при этом у него самый маленький показатель RMSE. Самый худший показатель RMSE - `40` у третьего региона.

## Подготовка к расчёту прибыли

In [21]:
# Бюджет
print('Бюджет:', BUDGET)

# Сколько нужно тысяч баррелей для того, чтобы окупить затраты
print('Необходимое количество для окупаемости:', COUNT_BARR, 'тыс. барр.')

# Средний истинный запас сырья в регионах, тыс. баррелей
data0_mean = 92.07859674082927
data1_mean = 68.72313602435997
data2_mean = 94.88423280885438
print('Объём сырья для безубыточной разработки новой скважины в первом регионе:', COUNT_BARR - data0_mean)
print('Объём сырья для безубыточной разработки новой скважины во втором регионе:', COUNT_BARR - data1_mean)
print('Объём сырья для безубыточной разработки новой скважины в третьем регионе:', COUNT_BARR - data2_mean)

Бюджет: 50000000.0
Необходимое количество для окупаемости: 111.11111111111111 тыс. барр.
Объём сырья для безубыточной разработки новой скважины в первом регионе: 19.032514370281845
Объём сырья для безубыточной разработки новой скважины во втором регионе: 42.38797508675114
Объём сырья для безубыточной разработки новой скважины в третьем регионе: 16.226878302256736


* Чтобы была окупаемость - необходимо выбрать скважины с запасом более 111 тыс. баррелей.
* Второй регион выглядит затратным

## Расчёт прибыли и рисков 

In [22]:
pd.options.mode.chained_assignment = None # Чтобы не появлялось предупреждение

In [23]:
# Cоздам функцию по расчету прибыли

# def profit(target, predictions, count):
#     profit = 0
#     probs_sorted = predictions.sort_values(ascending=False).head(200)
#     selected = target[probs_sorted.index][:count] 
#     for top in selected:
#         profit += (top - COUNT_BARR)*450000
#     return profit

In [24]:
def profit(target, predictions, count):
    profit = 0
    probs_sorted = predictions.sort_values(ascending=False).head(200)
    selected = target[probs_sorted.index][:count] 
    profit += ((selected - COUNT_BARR) * 450000).sum()
    
    return profit

In [25]:
data0_valid['product_pred'] = data0_predictions_valid
data0_valid.reset_index(drop=True, inplace=True)
profit0 = profit(data0_valid['product'], data0_valid['product_pred'], 200)
print('Прибиыльность 200 лучших скважин валидационной выборки', round(profit0/1000000), 'млн.')

Прибиыльность 200 лучших скважин валидационной выборки 3321 млн.


In [26]:
data1_valid['product_pred']=data1_predictions_valid
data1_valid.reset_index(drop=True, inplace=True)
profit1=profit(data1_valid['product'], data1_valid['product_pred'],200)
print('Прибиыльность 200 лучших скважин валидационной выборки', round(profit1/1000000), 'млн.')

Прибиыльность 200 лучших скважин валидационной выборки 2415 млн.


In [27]:
data2_valid['product_pred']=data2_predictions_valid
data2_valid.reset_index(drop=True, inplace=True)
profit2=profit(data2_valid['product'], data2_valid['product_pred'],200)
print('Прибиыльность 200 лучших скважин валидационной выборки', round(profit2/1000000), 'млн.')

Прибиыльность 200 лучших скважин валидационной выборки 2710 млн.


In [28]:
# Создадим функцию, которая найдет 500 случайных точек и выберет из них 200 лучших 1000 раз

def bootstrap (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        target_subsample.reset_index(drop=True, inplace=True)
        preds_subsample.reset_index(drop=True, inplace=True)
        values.append(profit(target_subsample, preds_subsample, 200))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
   
    
    print('Вероятность убытков',stats.percentileofscore(values, 0),'%')
    print('Среднее значение bootstrap', (values.mean())/1000000, 'млн.')
    print('Верхняя граница доверительного интервала', higher)
    print('Нижняя граница доверительного интервала', lower)
    print()
        
print('Для первого региона')
bootstrap(data0_valid['product'], data0_valid['product_pred'])
print()
print('Для второго региона')
bootstrap(data1_valid['product'], data1_valid['product_pred'])
print()
print('Для третьего региона')
bootstrap(data2_valid['product'], data2_valid['product_pred'])

Для первого региона
Вероятность убытков 6.9 %
Среднее значение bootstrap 396.16498480237084 млн.
Верхняя граница доверительного интервала 909766941.5534226
Нижняя граница доверительного интервала -111215545.89049557


Для второго региона
Вероятность убытков 1.5 %
Среднее значение bootstrap 456.04510578666026 млн.
Верхняя граница доверительного интервала 852289453.866034
Нижняя граница доверительного интервала 33820509.39898549


Для третьего региона
Вероятность убытков 7.6000000000000005 %
Среднее значение bootstrap 404.4038665683566 млн.
Верхняя граница доверительного интервала 950359574.9237998
Нижняя граница доверительного интервала -163350413.39560002



## Общий вывод:

* У первого региона больше всего скважин с лучшими запасами. Он показал прибыль в `3.4` млрд.
* У второго региона меньше всего вероятность понести убытки - всего `1.5%`
* Третий регион имеет наибольшую, среди остальных, верхнюю границу доверительного интервала.

На основании проведенного исследования отметим, что подходит под поставленные условия задачи бизнеса только второй регион, т.к. вероятность убытка только у него `>2.5%`